# MeLi Data Challenge 2021

## Imports

In [4]:
import pandas as pd

## Load Data

In [7]:
train_set = pd.read_parquet(r"https://meli-data-challenge.s3.amazonaws.com/2021/train_data.parquet")
train_set.head()

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.0


In [6]:
test_set = pd.read_csv(r"https://meli-data-challenge.s3.amazonaws.com/2021/test_data.csv")
test_set.head()

,sku,target_stock
0,464801,3
1,645793,4
2,99516,8
3,538100,8
4,557191,10
